# Youtube Downloader

test data: [Coding Bootcamp Berlin | Le Wagon Career Day](https://www.youtube.com/watch?v=JFsZxgy0Npw)

In [12]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   634  100   634    0     0   2193      0 --:--:-- --:--:-- --:--:--  2193
100  180M  100  180M    0     0  22.8M      0  0:00:07  0:00:07 --:--:-- 25.1M  0     0  21.6M      0  0:00:08  0:00:05  0:00:03 22.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   667  100   667    0     0   2479      0 --:--:-- --:--:-- --:--:--  2470
100  909M  100  909M    0     0  22.1M      0  0:00:41  0:00:41 --:--:-- 24.2M


In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from deepspeech import Model
from scipy.io.wavfile import read as wav_read
from pytube import YouTube
import os
import ffmpeg

In [29]:
! apt-get update && apt-get install -y libsndfile1 ffmpeg
! pip install Cython
! pip install nemo_toolkit

zsh:1: command not found: apt-get


In [32]:
import nemo
from nemo.collections.nlp.models import PunctuationCapitalizationModel
from transformers import AutoTokenizer, AutoModelWithLMHead

ModuleNotFoundError: No module named 'nemo'

In [44]:
def get_wave(url):
    text = (f'{url}')
    yt = YouTube(text)
    stream_url = yt.streams.all()[0].url
    audio, err = (ffmpeg.input(stream_url).output(
        "pipe:", format='wav', acodec='pcm_s16le').run(capture_stdout=True))
    with open('audio.wav', 'wb') as f:
        f.write(audio)
        audio_file = 'audio.wav'
        
        return audio_file

In [45]:
audio = get_wave('https://www.youtube.com/watch?v=JFsZxgy0Npw')

/var/folders/sy/lghshr2j337gznw_5lk00gn40000gn/T/ipykernel_34600/464524003.py:4: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  stream_url = yt.streams.all()[0].url
ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.3)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enab

In [39]:
def get_text(audio_file):
    model_file_path = "./deepspeech-0.9.3-models.pbmm"
    lm_file_path = "./deepspeech-0.9.3-models.scorer"
    beam_width = 3000
    lm_alpha = 0.931289039105002
    lm_beta = 1.1834137581510284

    model = Model(model_file_path)
    model.enableExternalScorer(lm_file_path)
    model.setScorerAlphaBeta(lm_alpha, lm_beta)
    model.setBeamWidth(beam_width)
    rate, buffer= wav_read(audio_file)
    row_text = model.stt(buffer)
    
    return row_text

In [46]:
row_text = get_text(audio)
row_text

TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858
/var/folders/sy/lghshr2j337gznw_5lk00gn40000gn/T/ipykernel_34600/959718576.py:12: WavFileWarning: Reached EOF prematurely; finished at 3889231 bytes, expected 4294967303 bytes from header.
  rate, buffer= wav_read(audio_file)


'to this interpretation trumpery certainly in for a more happy to be connected in beetween the two of the committee a behaving to change them ominously to woodland they all turned all to be of lasting importance to spare the totalisator the old botanist a battered learn more and all on what i and initiator with other people viperine i get a chance to make an impression and actually meeting close to belittle the fortifications the people that what is complete proposition to be a commissioner active proletarian possessed to all brotherhoods gianforte depositors table sederunt of companies of the stature of resistance proportions no i would never have considered to be before it the wisitor what my future was and what was the word is canalopolis'

SyntaxError: invalid syntax (3644523732.py, line 1)

In [ ]:
def get_sentence(unpunctuated_text):
    model = PunctuationCapitalizationModel.from_pretrained(model_name="punctuation_en_distilbert")
    punctuated_text = model.add_punctuation_capitalization([unpunctuated_text])
    
    return punctuated_text   

In [ ]:
punctuated_text = get_sentence(row_text)
puctuated_text

In [ ]:
def get_correction(row_sentence):
    tokenizer = AutoTokenizer.from_pretrained("flexudy/t5-base-multi-sentence-doctor")
    model = AutoModelWithLMHead.from_pretrained("flexudy/t5-base-multi-sentence-doctor")

    input_ids = tokenizer.encode(row_sentence, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=32, num_beams=1)

sentence = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

assert sentence == "I am a medical doctor."
    

In [ ]:
corrected_text = get_correction(punctuated_text)